In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

W0222 00:18:34.833594 4403418560 deprecation.py:323] From <ipython-input-1-8bf8ae5a5303>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0222 00:18:34.835017 4403418560 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0222 00:18:34.836045 4403418560 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0222 00:18:35.065598 4403418560 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf
import numpy as np
np.set_printoptions(suppress=True) 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [3]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1./tf.sqrt(in_dim/2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

mb_size = 32
X_dim = mnist.train.images.shape[1]
Z_dim = 16
c_dim = 10

X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob

Z = tf.placeholder(tf.float32, shape=[None, Z_dim])
c = tf.placeholder(tf.float32, shape=[None, c_dim])

G_W1 = tf.Variable(xavier_init([Z_dim + c_dim, 256]))
G_b1 = tf.Variable(tf.zeros(shape=[256]))

G_W2 = tf.Variable(xavier_init([256, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

def generator(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

Q_W1 = tf.Variable(xavier_init([X_dim, 128]))
Q_b1 = tf.Variable(tf.zeros(shape=[128]))

Q_W2 = tf.Variable(xavier_init([128, c_dim]))
Q_b2 = tf.Variable(tf.zeros(shape=[c_dim]))

theta_Q = [Q_W1, Q_W2, Q_b1, Q_b2]

def Q(x):
    Q_h1 = tf.nn.relu(tf.matmul(x, Q_W1) + Q_b1)
    Q_prob = tf.nn.softmax(tf.matmul(Q_h1, Q_W2) + Q_b2)
    return Q_prob

def sample_Z(m,n):
    return np.random.uniform(-1., 1., size=[m,n])

def sample_c(m):
    return np.random.multinomial(1, 10*[0.1], size=m)

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

G_sample = generator(Z, c)
D_real = discriminator(X)
D_fake = discriminator(G_sample)
Q_c_given_x = Q(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real + 1e-8) + tf.log(1. - D_fake + 1e-8))
G_loss = -tf.reduce_mean(tf.log(D_fake + 1e-8))

cross_ent = tf.reduce_mean(-tf.reduce_sum(tf.log(Q_c_given_x + 1e-8)*c, 1))
ent = tf.reduce_mean(-tf.reduce_sum(tf.log(c + 1e-8)*c, 1))
Q_loss = cross_ent + ent

# D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
# D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
# D_loss = D_loss_real + D_loss_fake
# G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)
Q_solver = tf.train.AdamOptimizer().minimize(Q_loss, var_list=theta_G + theta_Q)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')
    
i=0
for it in range(1000000):
    if it%10000==0:
        for j in range(10):
            n_sample = 16

            Z_sample = sample_Z(n_sample, Z_dim)
#             idx = np.random_randint(0, 10)
            c_sample = np.zeros([n_sample, c_dim])
#             c_noise[range(n_sample), idx] = 1
            c_sample[:, j] = 1

            samples = sess.run(G_sample, feed_dict={Z:Z_sample, c:c_sample})

            fig = plot(samples)
            plt.savefig('out/{}.png'.format((str(i)+'_'+str(j)).zfill(3)), bbox_inches='tight')
            i+=1
            plt.close(fig)
        
    X_mb, _ =  mnist.train.next_batch(mb_size)
    Z_sample = sample_Z(mb_size, Z_dim)
    c_sample = sample_c(mb_size)
        
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X:X_mb, Z:Z_sample, c:c_sample})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z:Z_sample, c:c_sample})
    
    sess.run([Q_solver], feed_dict={Z:Z_sample, c:c_sample})
    
    if it%1000==0:
        print('iter:{}'.format(it))
        print('D loss:{:.4}'.format(D_loss_curr))
        print('G loss:{:.4}'.format(G_loss_curr))
        print()

W0222 00:18:35.475564 4403418560 deprecation.py:506] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/python/training/slot_creator.py:187: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


iter:0
D loss:1.302
G loss:2.638

iter:1000
D loss:0.09613
G loss:4.398

iter:2000
D loss:0.1205
G loss:4.196

iter:3000
D loss:0.3335
G loss:4.137

iter:4000
D loss:0.08644
G loss:4.44

iter:5000
D loss:0.331
G loss:3.395

iter:6000
D loss:0.4869
G loss:4.505

iter:7000
D loss:0.2974
G loss:3.35

iter:8000
D loss:0.4213
G loss:3.121

iter:9000
D loss:0.7616
G loss:3.086

iter:10000
D loss:0.6257
G loss:3.331

iter:11000
D loss:0.5748
G loss:3.218

iter:12000
D loss:0.8975
G loss:2.226

iter:13000
D loss:0.7327
G loss:1.893

iter:14000
D loss:1.235
G loss:1.852

iter:15000
D loss:0.6559
G loss:2.334

iter:16000
D loss:0.9213
G loss:1.831

iter:17000
D loss:0.7477
G loss:1.695

iter:18000
D loss:0.8449
G loss:1.717

iter:19000
D loss:0.9688
G loss:1.477

iter:20000
D loss:0.8979
G loss:1.832

iter:21000
D loss:1.031
G loss:1.261

iter:22000
D loss:0.8389
G loss:1.399

iter:23000
D loss:1.046
G loss:1.469

iter:24000
D loss:0.8749
G loss:1.602

iter:25000
D loss:1.047
G loss:1.482

iter:

iter:212000
D loss:1.233
G loss:1.332

iter:213000
D loss:0.8392
G loss:1.37

iter:214000
D loss:0.8765
G loss:1.221

iter:215000
D loss:0.9007
G loss:1.354

iter:216000
D loss:1.161
G loss:1.593

iter:217000
D loss:0.9887
G loss:1.186

iter:218000
D loss:0.919
G loss:1.448

iter:219000
D loss:1.161
G loss:1.304

iter:220000
D loss:0.7596
G loss:1.332

iter:221000
D loss:1.14
G loss:1.243

iter:222000
D loss:1.124
G loss:1.208

iter:223000
D loss:0.9643
G loss:1.337

iter:224000
D loss:1.014
G loss:1.088

iter:225000
D loss:0.9314
G loss:1.292

iter:226000
D loss:0.8592
G loss:1.559

iter:227000
D loss:0.8601
G loss:1.46

iter:228000
D loss:0.7844
G loss:1.498

iter:229000
D loss:0.835
G loss:1.43

iter:230000
D loss:1.154
G loss:1.295

iter:231000
D loss:1.224
G loss:1.366

iter:232000
D loss:1.159
G loss:1.347

iter:233000
D loss:0.9973
G loss:1.269

iter:234000
D loss:1.212
G loss:1.139

iter:235000
D loss:0.9
G loss:1.548

iter:236000
D loss:0.8943
G loss:1.297

iter:237000
D loss:

iter:420000
D loss:1.031
G loss:1.184

iter:421000
D loss:0.9175
G loss:1.591

iter:422000
D loss:1.04
G loss:1.459

iter:423000
D loss:0.6854
G loss:1.614

iter:424000
D loss:0.9576
G loss:1.486

iter:425000
D loss:0.873
G loss:1.822

iter:426000
D loss:0.7482
G loss:1.495

iter:427000
D loss:1.069
G loss:1.666

iter:428000
D loss:0.9927
G loss:1.371

iter:429000
D loss:0.8801
G loss:1.421

iter:430000
D loss:0.6918
G loss:1.825

iter:431000
D loss:1.079
G loss:1.475

iter:432000
D loss:1.015
G loss:1.362

iter:433000
D loss:1.194
G loss:1.469

iter:434000
D loss:0.8029
G loss:1.575

iter:435000
D loss:1.035
G loss:1.562

iter:436000
D loss:0.927
G loss:1.483

iter:437000
D loss:1.143
G loss:1.443

iter:438000
D loss:0.7742
G loss:1.574

iter:439000
D loss:0.9979
G loss:1.388

iter:440000
D loss:0.9889
G loss:1.483

iter:441000
D loss:0.9163
G loss:1.511

iter:442000
D loss:0.8181
G loss:1.566

iter:443000
D loss:0.8508
G loss:1.725

iter:444000
D loss:0.8551
G loss:1.318

iter:445000

iter:628000
D loss:0.9394
G loss:1.207

iter:629000
D loss:0.8661
G loss:1.281

iter:630000
D loss:0.8833
G loss:1.328

iter:631000
D loss:1.116
G loss:1.415

iter:632000
D loss:0.8912
G loss:1.386

iter:633000
D loss:0.7897
G loss:1.581

iter:634000
D loss:0.8657
G loss:1.228

iter:635000
D loss:0.8
G loss:1.717

iter:636000
D loss:0.9891
G loss:1.919

iter:637000
D loss:0.8548
G loss:1.533

iter:638000
D loss:0.9718
G loss:1.293

iter:639000
D loss:0.913
G loss:1.813

iter:640000
D loss:1.014
G loss:1.215

iter:641000
D loss:1.007
G loss:1.527

iter:642000
D loss:0.8988
G loss:1.521

iter:643000
D loss:1.059
G loss:2.055

iter:644000
D loss:0.7722
G loss:1.605

iter:645000
D loss:1.112
G loss:1.176

iter:646000
D loss:1.086
G loss:1.652

iter:647000
D loss:0.7403
G loss:2.098

iter:648000
D loss:0.5684
G loss:1.829

iter:649000
D loss:0.9561
G loss:1.301

iter:650000
D loss:0.8516
G loss:1.41

iter:651000
D loss:0.7612
G loss:1.65

iter:652000
D loss:0.8777
G loss:1.257

iter:653000


iter:836000
D loss:0.948
G loss:1.515

iter:837000
D loss:1.205
G loss:1.36

iter:838000
D loss:0.6445
G loss:1.669

iter:839000
D loss:1.025
G loss:1.569

iter:840000
D loss:0.9014
G loss:1.375

iter:841000
D loss:0.8323
G loss:1.817

iter:842000
D loss:0.724
G loss:1.388

iter:843000
D loss:0.9308
G loss:2.045

iter:844000
D loss:1.102
G loss:1.168

iter:845000
D loss:0.8179
G loss:1.333

iter:846000
D loss:0.7655
G loss:1.367

iter:847000
D loss:1.14
G loss:1.275

iter:848000
D loss:0.8325
G loss:1.746

iter:849000
D loss:1.037
G loss:1.249

iter:850000
D loss:0.7449
G loss:1.761

iter:851000
D loss:0.8981
G loss:1.406

iter:852000
D loss:0.8597
G loss:1.474

iter:853000
D loss:0.8948
G loss:1.389

iter:854000
D loss:1.032
G loss:1.409

iter:855000
D loss:0.7685
G loss:1.464

iter:856000
D loss:0.844
G loss:1.857

iter:857000
D loss:0.7905
G loss:1.623

iter:858000
D loss:0.7382
G loss:1.522

iter:859000
D loss:1.171
G loss:1.389

iter:860000
D loss:1.027
G loss:1.027

iter:861000
D